In [ ]:
# _3-30pm

if long_short_both == False: # there's only one trading direction:
        
            for i, (ts, signl) in enumerate(signal_updates.items()):
                if i:
                    each_profit = PortfolioCalculation.profit(init_ticker_price, ticker_price_updates[ts], init_ticker_qty, strategy_drct)
                else:
                    each_profit = 0  
                init_ticker_price = ticker_price_updates[ts]
                
                if signl != init_signal:   #signal starts to change, meaning we need to enter or exit positions
                    init_signal = signl
                    
                    if signl == enter_signal:  #only need to update position's qty here
                        updated_qty = PortfolioCalculation.quantity(self.tradable_capital, ticker_price_updates[ts], self.port_utilization) ################## 'ticker_qty' that the commission package needs
                        entered = True
                        init_ticker_qty = updated_qty 
                        
                        ################# Adding first trade stats: cumulatie trades in total:
                        cum_trades = 1 + cum_trades
                        ################# Adding second trade stats: commission fee:
                        each_commission_cost = ib_equity_commissions(updated_qty, ticker_price_updates[ts])
                        ################# Adding third trade stats: volume_impact_pct:
                        volume_impact_pct = init_ticker_qty/signal_dataframe['total_quantity'][ts]
                        ################# Adding fourth trade stats: 
                        filled_price_impact_pct = (self.price_impact_constant)*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*(init_ticker_qty/signal_dataframe['total_quantity'][ts])
                        ################# Adding fifth trade stats: 
                        filled_price_impact_abs = (self.price_impact_constant)*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*init_ticker_price
                        ################# Adding sixth trade stats:
                        filled_price_impact_chg_dlr = filled_price_impact_abs if strategy_drct == 'L' else (-filled_price_impact_abs if strategy_drct == 'S' else 0) 
                        ################# Adding seventh trade stats:
                        actual_filled_price = filled_price_impact_chg_dlr + init_ticker_price
                        
                        
                        
                    elif signl == exit_signal:
                        updated_qty = PortfolioCalculation.quantity(self.tradable_capital, ticker_price_updates[ts], self.port_utilization) ################## 'ticker_qty' that the commission package needs
                        exited = True
                        init_ticker_qty = updated_qty 
                        ################# Adding first trade stats: cumulatie trades in total:
                        cum_trades = 1 + cum_trades
                        ################# Adding second trade stats: commission fee:
                        each_commission_cost = ib_equity_commissions(updated_qty, ticker_price_updates[ts])
                        ################# Adding third trade stats: volume_impact_pct:
                        volume_impact_pct = init_ticker_qty/signal_dataframe['total_quantity'][ts]
                        ################# Adding fourth trade stats: 
                        filled_price_impact_pct = (self.price_impact_constant)*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*(init_ticker_qty/signal_dataframe['total_quantity'][ts])
                        ################# Adding fifth trade stats: 
                        filled_price_impact_abs = (self.price_impact_constant)*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*init_ticker_price
                        ################# Adding sixth trade stats:
                        filled_price_impact_chg_dlr = filled_price_impact_abs if strategy_drct == 'L' else (-filled_price_impact_abs if strategy_drct == 'S' else 0) 
                        ################# Adding seventh trade stats:
                        actual_filled_price = filled_price_impact_chg_dlr + init_ticker_price
                        

                # After signal changes, after looping through each row, we need to update stock price:
                

                if exited:             # if there's no position, update qty back to 0
                    init_ticker_qty = 0
                    exited = False     # update exit status back to initialization of being False

                    
                each_row = {
                    'timestamp':ts, # save updated index
                    'ticker_price':ticker_price_updates[ts], # save updated price
                    'signal':init_signal, # save previous signal
                    'ticker_qty':int(init_ticker_qty), # save updated qty
                    'profit':each_profit, # save updated profit
                    
                    ################## Add more details regarding the trading stats and market impact.
                    'cumulative_trades':cum_trades, 
                    'commissions':each_commission_cost,
                    'mkt_qty_available':signal_dataframe['total_quantity'][ts],
                    'mkt_traded_volume':signal_dataframe['total_volume'][ts],
                    'volume_impact_pct':volume_impact_pct,
                    'filled_price_impact_pct':filled_price_impact_pct,
                    'filled_price_impact_abs':filled_price_impact_abs,
                    'filled_price_impact_chg_dlr':filled_price_impact_chg_dlr, 
                    'actual_filled_price': actual_filled_price
                }

                self.data.append(each_row)
                 # At last, update the signal_0 to this row's signal, which is 'signl'

            data_df = pd.DataFrame(self.data)
            data_df.index = data_df['timestamp']

            # Connect data_df with the later df2:



            # copy df1 to df2, creating a new dataframe for storing tearsheet data for printing
            df2 = data_df
            df2['each_profit'] = data_df['profit']
            df2['cum_profit'] = data_df.profit.cumsum() + self.tradable_capital
            df2['each_return_pct'] = df2.cum_profit.pct_change().fillna(0)
            df2['cum_return_pct'] = (df2['each_return_pct'] + 1).cumprod()

            df2 = df2.dropna(0)

In [ ]:
# 4:30

    def core_backtester(self, signal_dataframe, df_sgnl_str, df_tkr_price_str, strategy_drct, 
                        long_short_both, enter_signal, exit_signal, long_signal, short_signal):
        
        signal_updates = signal_dataframe[df_sgnl_str]
        ticker_price_updates = signal_dataframe[df_tkr_price_str]
        init_signal = 0
        init_ticker_qty = 0
        init_port_value = 0 ############# maybe we can delete this init_port_value
        init_ticker_price = 0
        init_mkt_qty = 0
        init_mkt_volume = 0
        cum_trades = 0
        cum_commissions = 0
        exited = False
        
        if long_short_both == False: # there's only one trading direction:
        
            for i, (ts, signl) in enumerate(signal_updates.items()):
                if i:
                    each_profit = PortfolioCalculation.profit(init_ticker_price, ticker_price_updates[ts], init_ticker_qty, strategy_drct)
                else:
                    each_profit = 0  
                init_ticker_price = ticker_price_updates[ts]
                
                if signl != init_signal:   #signal starts to change, meaning we need to enter or exit positions
                    init_signal = signl
                    
                    if signl == enter_signal:  #only need to update position's qty here
                        updated_qty = PortfolioCalculation.quantity(self.tradable_capital, ticker_price_updates[ts], self.port_utilization) ################## 'ticker_qty' that the commission package needs
                        entered = True
                        init_ticker_qty = updated_qty 
                        
                        ################# Adding first trade stats: cumulatie trades in total:
                        cum_trades = 1 + cum_trades
                        ################# Adding third trade stats: volume_impact_pct:
                        volume_impact_pct = init_ticker_qty/signal_dataframe['total_quantity'][ts]
                        ################# Adding fourth trade stats: 
                        filled_price_impact_pct = (self.price_impact_constant)*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*(init_ticker_qty/signal_dataframe['total_quantity'][ts])
                        ################# Adding fifth trade stats: 
                        filled_price_impact_abs = (self.price_impact_constant)*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*init_ticker_price
                        ################# Adding sixth trade stats:
                        filled_price_impact_chg_dlr = filled_price_impact_abs if strategy_drct == 'L' else (-filled_price_impact_abs if strategy_drct == 'S' else 0) 
                        ################# Adding seventh trade stats:
                        actual_filled_price = filled_price_impact_chg_dlr + init_ticker_price
                        
                        
                        
                    elif signl == exit_signal:
                        updated_qty = PortfolioCalculation.quantity(self.tradable_capital, ticker_price_updates[ts], self.port_utilization) ################## 'ticker_qty' that the commission package needs
                        exited = True
                        init_ticker_qty = updated_qty 
                        ################# Adding first trade stats: cumulatie trades in total:
                        cum_trades = 1 + cum_trades
                        
                        ################# Adding third trade stats: volume_impact_pct:
                        volume_impact_pct = init_ticker_qty/signal_dataframe['total_quantity'][ts]
                        ################# Adding fourth trade stats: 
                        filled_price_impact_pct = (self.price_impact_constant)*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*(init_ticker_qty/signal_dataframe['total_quantity'][ts])
                        ################# Adding fifth trade stats: 
                        filled_price_impact_abs = (self.price_impact_constant)*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*(init_ticker_qty/signal_dataframe['total_quantity'][ts])*init_ticker_price
                        ################# Adding sixth trade stats:
                        filled_price_impact_chg_dlr = filled_price_impact_abs if strategy_drct == 'L' else (-filled_price_impact_abs if strategy_drct == 'S' else 0) 
                        ################# Adding seventh trade stats:
                        actual_filled_price = filled_price_impact_chg_dlr + init_ticker_price
                        

                # After signal changes, after looping through each row, we need to update stock price:
                    ################# Adding second trade stats: commission fee:
                    each_commission_cost = ib_equity_commissions(init_ticker_qty, ticker_price_updates[ts])
                    cum_commissions += each_commission_cost

                if exited:             # if there's no position, update qty back to 0
                    init_ticker_qty = 0
                    exited = False     # update exit status back to initialization of being False, cannot change to True or the traded qty in position will be all 0

                each_row = {
                    'timestamp':ts, # save updated index
                    'ticker_price':ticker_price_updates[ts], # save updated price
                    'signal':init_signal, # save previous signal
                    'ticker_qty':int(init_ticker_qty), # save updated qty
                    'profit':each_profit, # save updated profit
                    
                    ################## Add more details regarding the trading stats and market impact.
                    'cumulative_trades':cum_trades, 
                    'commissions':each_commission_cost,
                    'cum_commissions': cum_commissions,
                    'mkt_qty_available':signal_dataframe['total_quantity'][ts],
                    'mkt_traded_volume':signal_dataframe['total_volume'][ts],
                    'volume_impact_pct':volume_impact_pct,
                    'filled_price_impact_pct':filled_price_impact_pct,
                    'filled_price_impact_abs':filled_price_impact_abs,
                    'filled_price_impact_chg_dlr':filled_price_impact_chg_dlr, 
                    'actual_filled_price': actual_filled_price
                }

                self.data.append(each_row)
                 # At last, update the signal_0 to this row's signal, which is 'signl'

            data_df = pd.DataFrame(self.data)
            data_df.index = data_df['timestamp']

            # Connect data_df with the later df2:



            # copy df1 to df2, creating a new dataframe for storing tearsheet data for printing
            df2 = data_df
            df2['each_profit'] = data_df['profit']
            df2['cum_profit'] = data_df.profit.cumsum() + self.tradable_capital
            df2['each_return_pct'] = df2.cum_profit.pct_change().fillna(0)
            df2['cum_return_pct'] = (df2['each_return_pct'] + 1).cumprod()

            df2 = df2.dropna(0)